# 🏡 California Housing Dataset
Este notebook demuestra cómo automatizar la descarga de un conjunto de datos desde internet, descomprimirlo y cargarlo en un DataFrame para su análisis.

**Contexto:**
Los datos que veremos a continuación fueron extraidos del censo de California de 1990. El conjunto de datos contiene información sobre el precio de las viviendas en California, así como características demográficas y socioeconómicas de las áreas. Algunas de las variables son: población, ingresos, número de habitaciones, etc.

## 1. Importar librerías necesarias

In [ ]:
# PEP8 es una guía de estilo para escribir código Python. (https://www.python.org/dev/peps/pep-0008/)

# Librerías estándar de python
import os
import tarfile
import urllib.request as request

# Librerías de terceros
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Librerías locales


%matplotlib inline

## 2. Definir rutas y URL del dataset

In [ ]:
# Las constantes son variables que no deberían cambiar su valor a lo largo del programa.
# Por convención, se escriben en mayúsculas y con guiones bajos para separar palabras.

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# Creación de constante de forma alternativa:
# HOUSING_URL = f"{DOWNLOAD_ROOT}datasets/housing/housing.tgz"

## 3. Extracción de los datos

In [ ]:
def fetch_housing_data(housing_url: str = HOUSING_URL, housing_path: str = HOUSING_PATH) -> None:
    # Validamos que el directorio de destino exista, y si no, lo creamos.
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)

    # Creamos una variables para almacenar la ruta del archivo comprimido.
    tgz_path = os.path.join(housing_path, "housing.tgz")

    # Descargamos el archivo comprimido desde la URL proporcionada y lo guardamos en la ruta especificada.
    request.urlretrieve(housing_url, tgz_path)

    # Abrimos el archivo comprimido y extraemos su contenido en el directorio especificado.
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

    # Solución alternativa para extraer el contenido del archivo comprimido. Utilizando context manager.
    # with tarfile.open(tgz_path, "r:gz") as f:
    #    f.extractall(path=housing_path)

### 4.1. Definición de la función `load_housing_data()`
Esta función abre el archivo CSV descargado y lo convierte en un DataFrame de pandas para facilitar su análisis.

In [ ]:
# Llamamos a la función para descargar los datos
fetch_housing_data()

A continuación, cargamos los datos y mostramos las primeras filas del DataFrame para verificar que la lectura ha sido exitosa.

In [ ]:
# Podemos listar los archivos que se han descargado para verificar que todo está correcto.
!ls -l {HOUSING_PATH}

## 4. Cargar los datos

In [ ]:
def load_housing_data(housing_path: str = HOUSING_PATH) -> pd.DataFrame:
    # Definimos una variable que almacena la ruta del archivo CSV dentro del directorio de datos.
    csv_path = os.path.join(housing_path, "housing.csv")
    # Leemos el archivo CSV y lo convertimos en un DataFrame de pandas.
    return pd.read_csv(csv_path)

### 5. Inspección general de los datos
Usamos el método `.info()` para conocer el número de columnas, tipos de datos, y si existen valores faltantes.

In [ ]:
# Llamamos a la función para cargar los datos en un DataFrame de pandas.
# El resultado de la función lo almacenamos en una variable llamada `housing`.
housing = load_housing_data()

# Presentamos las primeras 5 filas del DataFrame para verificar que los datos se han cargado correctamente.
housing.head()

## 5. EDA: Exploración de Datos

### 5.1 Tipos de datos

In [ ]:
# El método `info()` nos proporciona un resumen del DataFrame, incluyendo el número de entradas, tipos de datos y valores no nulos.
housing.info()

Usamos `.describe()` para obtener estadísticas descriptivas como media, desviación estándar, mínimo y máximo por columna numérica.

In [ ]:
housing["ocean_proximity"].value_counts()

### 5.2 Estadísticas descriptivas

In [ ]:
# El método `describe()` nos proporciona estadísticas descriptivas de las columnas numéricas del DataFrame.
housing.describe()

### 5.3 Histograma de las variables numéricas

In [ ]:
# El método `hist()` nos permite visualizar la distribución de los datos numéricos en el DataFrame.
housing.hist(bins=50, figsize=(20, 15))
plt.show()

### 5.1. Visualización inicial
Graficamos histogramas de las variables numéricas para ver su distribución y detectar posibles valores extremos.

In [ ]:
# Lista de las columnas del DataFrame `housing`.
housing.columns

## 🧪 6. Preparación de los datos
En esta sección prepararemos los datos para un futuro modelo de machine learning. Esto incluye manejo de valores faltantes, codificación de variables categóricas, y separación de variables predictoras y objetivo.

### 6.1 Dividir el conjunto de datos entre Train y Test

In [ ]:
housing_train, housing_test = train_test_split(housing, test_size=0.2, random_state=42, shuffle=False)

### 6.1. Identificación de valores faltantes
Contamos los valores nulos por columna para saber dónde debemos aplicar imputación.

In [ ]:
housing_train.head()

Aquí observamos que `total_bedrooms` tiene valores faltantes. Vamos a imputarlos con la mediana, una estrategia común.

In [ ]:
housing_train.shape

Obtenemos la mediana de la columna.

In [ ]:
housing_test.shape

Rellenamos los valores nulos de `total_bedrooms` usando esa mediana.

In [ ]:
housing = housing_train.copy()

Verificamos que ya no hay valores nulos en `total_bedrooms`.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

Confirmamos que el DataFrame ya no contiene valores faltantes.

In [ ]:
corr_matrix = housing.loc[:, housing.columns != "ocean_proximity"].corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

### 6.2. Codificación de variables categóricas
Convertimos la columna `ocean_proximity` en variables numéricas utilizando codificación one-hot con `pd.get_dummies()`.

In [ ]:
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.figure(figsize=(10, 8))
plt.show()

### 6.2 Ingeniería de características

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

### 6.3. Separación de variables predictoras y objetivo
Separaremos nuestro dataset en `X` (las columnas que usaremos para predecir) y `y` (la variable que queremos predecir: `median_house_value`).

In [ ]:
corr_matrix = housing.loc[:, housing.columns != "ocean_proximity"].corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

### 6.3 Limpieza de datos

In [ ]:
housing = housing_train.drop("median_house_value", axis=1)
housing_labels = housing_train["median_house_value"].copy()

#### 6.3.1. Manejo de valores faltantes
Primero identificamos columnas con valores faltantes. Luego decidiremos si los eliminamos o los imputamos (rellenamos).

In [ ]:
# Contamos cuántos valores faltantes hay en cada columna
housing.isnull().sum()

### 6.4. Escalado de variables numéricas
Escalamos las columnas numéricas con `StandardScaler` para que todas tengan media 0 y desviación estándar 1.

In [ ]:
housing[housing["total_bedrooms"].isnull()]

Vemos que la columna `total_bedrooms` contiene valores faltantes. Vamos a reemplazarlos con la **mediana** de esa columna.

In [ ]:
# Vamos a obtener la mediana de la columna "total_bedrooms" para rellenar los valores faltantes.
imputer = SimpleImputer(strategy="median")

Aplicamos el escalado sobre los datos numéricos.

In [ ]:
# Dado que la imputación de datos solo se puede aplicar a columnas numéricas, vamos a crear un nuevo DataFrame que contenga solo las columnas numéricas.
housing_num = housing.drop("ocean_proximity", axis=1)

Convertimos la salida escalada a un DataFrame de pandas con nombres de columnas.

In [ ]:
#
imputer.fit(housing_num)

imputer.statistics_

Identificamos las columnas categóricas que ya han sido codificadas.

In [ ]:
X = imputer.transform(housing_num)

housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
housing_tr.head()

Unimos las columnas numéricas escaladas con las categóricas codificadas para formar `X_preparado`, listo para entrenamiento.

In [ ]:
# Validamos que ya no hay valores faltantes en la columna "total_bedrooms".
assert not housing_tr["total_bedrooms"].isnull().any(), "Hay valores faltantes en total_bedrooms"

Mostramos las primeras filas del dataset final preparado.

In [ ]:
# Mostramos la información del DataFrame para verificar que los cambios se han aplicado correctamente.
housing_tr.info()

#### 6.3.2. Codificación de variables categóricas
La columna `ocean_proximity` es de tipo texto (categórica). Vamos a convertirla en variables numéricas usando codificación *one-hot*.

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

### 6.5. Selección de variables con OLS (Ordinary Least Squares)
Ajustamos un modelo de regresión lineal para ver qué variables son más significativas al predecir el valor de la vivienda.

In [ ]:
encoder = OrdinalEncoder()
housing_cat_encoded = encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

Usamos `statsmodels` para entrenar el modelo con todas las variables, incluyendo una constante.

In [ ]:
encoder.categories_

Mostramos el resumen del modelo OLS con coeficientes, errores estándar, R² y p-values.

In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()[:10]

### 6.6. Filtrar variables significativas
Seleccionamos las variables cuyo p-value sea menor a 0.05 para construir un subconjunto más relevante.

In [ ]:
housing_cat_1hot = pd.DataFrame(housing_cat_1hot.toarray(), columns=cat_encoder.get_feature_names_out(), index=housing_cat.index)

#### 6.3.3. Escalar las variables numéricas

El escalado es importante para que todas las variables numéricas tengan un rango comparable, especialmente si vamos a usar modelos sensibles a magnitudes como regresión lineal o redes neuronales.

In [ ]:
scaler = StandardScaler()
housing_num_tr = scaler.fit_transform(housing_tr)

Extraemos los nombres de variables con significancia estadística.

In [ ]:
pd.DataFrame(housing_num_tr, columns=housing_tr.columns, index=housing_tr.index).head()

Creamos un nuevo DataFrame `X_filtrado` con solo esas columnas.

In [ ]:
housing_num_tr = pd.DataFrame(housing_num_tr, columns=housing_tr.columns, index=housing_tr.index)

Mostramos las primeras filas del nuevo dataset filtrado.

In [ ]:
# Combinamos numéricas escaladas + categóricas codificadas
housing_prepared = pd.concat([housing_num_tr, housing_cat_1hot], axis=1)
housing_prepared.head()

### 6.4 Uso de `Pipeline` de Scikit-learn

Usaremos un `Pipeline` para encadenar los pasos de preprocesamiento y entrenamiento del modelo. Esto nos permite aplicar transformaciones de manera ordenada y reproducible.

In [ ]:
num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])

housing_num_tr = num_pipeline.fit_transform(housing_num)

Ahora nuestro dataset está limpio, escalado, codificado y filtrado con variables relevantes listas para modelado.

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs), ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

### 7. Selección y entrenamiento del modelo

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

Verificamos la forma final de `X_filtrado`.

In [ ]:
datos_in = housing.iloc[:5]
datos_out = housing_labels.iloc[:5]

datos_preparados = full_pipeline.transform(datos_in)
predicciones = lin_reg.predict(datos_preparados)

print("Predicciones:", predicciones)

Confirmamos que `y` sigue alineado correctamente.

In [ ]:
print("Reales:", list(datos_out))

### 8. Evaluación del modelo

En esta sección evaluaremos el rendimiento del modelo utilizando métricas como RMSE (Raíz del Error Cuadrático Medio) y R² (Coeficiente de Determinación).

In [ ]:
predictions = lin_reg.predict(housing_prepared)
mse = mean_squared_error(housing_labels, predictions)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

🎉 ¡Listo! Hemos preparado un dataset completamente procesado para entrenar modelos de machine learning.

In [ ]:
r2 = r2_score(housing_labels, predictions)
print("R^2:", r2)

Entendiendo las métricas:
- **RMSE**: Mide el error promedio entre las predicciones del modelo y los valores reales.
    - Se calcula como la raíz cuadrada del error cuadrático medio (MSE).
    - Interpretación: valores más bajos indican mejor desempeño.
    - Está en la misma unidad que la variable objetivo (median_house_value en este caso).
- **R²**: Indica qué porcentaje de la variabilidad en los datos es explicado por el modelo.
    - Su valor va de 0 a 1 (en algunos casos puede ser negativo).
    - Un valor de 0.85 significa que el modelo explica el 85% de la variabilidad de los datos.